In [8]:
import pandas as pd
import networkx as nx
import nltk
from nltk.corpus import wordnet as wn
from tqdm import tqdm
import os
import glob
import numpy as np
import nltk


In [25]:
def clean(word):
    #word = word.split('.')[0].replace('_', ' ')
   # word = word.replace('_', ' ')
    word = word.replace('а́', 'а')
    word = word.replace('ё', "е")
    word = word.replace('у́', 'у')
    word = word.replace('о́', 'о')
    word = word.replace('и́', "и")
    word = word.replace('е́', "е")
    word = word.replace('я́', "я")

    if '(' in word and ')' in word:
        lb = word.find('(')
        rb = word.find(')')
        word = word[:lb-1] + word[rb+1:]
        word = word.strip()


    return word

def create_all_edges(parents, children):
    res = []
    
    for parent in parents:
        for child in children:
            res.append((parent, child))

    return res

def generator(G):
    while not nx.is_directed_acyclic_graph(G):
        cycle = nx.find_cycle(G)
        yield cycle[0]
        
def make_clean_graph(in_path, out_path):
    df = pd.read_csv(in_path, sep='\t')
    df1 = df[~df['WordnetKey'].isna()]
    df2 = df1[~df1['Definition'].isna()]

    G = nx.from_pandas_edgelist(df2, 'Hypernym',
                                 'Synset', create_using=nx.DiGraph)

    nodes_to_add = []
    edges_to_add = []
    nodes_to_delete = []

    for node in G:
        if len(str(node).split('\t')) > 1:
            nodes_to_add.extend((str(node).split('\t')))
            edges_to_add.extend(create_all_edges(str(node).split('\t'), list(G[node])))
            nodes_to_delete.append(node)

    G.add_nodes_from(nodes_to_add)
    G.add_edges_from(edges_to_add)
    G.remove_nodes_from(nodes_to_delete)

    for bad_edge in tqdm(generator(G)):
        G.remove_edge(*bad_edge)

    nx.write_edgelist(G, out_path, delimiter='\t')

In [17]:
# global_train = []
# global_test = []

# all_data = glob.glob(os.path.join("data/full_babel/", "*babel_hyper.csv"))
# for path in tqdm(all_data):
#     lang = path.split("/")[-1].replace("_babel_hyper.csv", "")
#     save_path = 'data/wnet_babel/{}_isa.edgelist'.format(lang)

#     make_clean_graph(path, save_path)

In [18]:
df = pd.read_csv('data/full_babel/ru_babel_hyper.csv', sep='\t')

In [28]:
for i, synset in tqdm(enumerate(df['Synset'])):
    df['Synset'][i] = clean(df['Synset'][i])
    all_hypernyms = df['Hypernym'][i]

    if not pd.isna(all_hypernyms):

        hyps = []
        for hyp in all_hypernyms.split('\t'):
            hyps.append(clean(hyp)) 

        df['Hypernym'][i] = '\t'.join(hyps)

0it [00:00, ?it/s]

69466it [00:42, 1621.97it/s]


In [33]:
df1 = df[~df['WordnetKey'].isna()]

In [34]:
df1.sample(20)

,Synset,Hypernym,WordnetKey,Definition
53612,солонка.n.3,NaN,wn:04131690n,NaN
4196,берберы.n.1,меньшинство.n.1,wn:08483500n,Бербе́ры (от греч.
16193,изменник.n.1,изменник.n.1,wn:09935793n,NaN
38043,втулка.n.6,подшипник.n.6,wn:03602686n,"Втулка — деталь машины, механизма, прибора или..."
61705,рокусьо.n.1,патина.n.1,wn:04528079n,Патина
32673,гольф_вдовы.n.1,жена.n.1,wn:10137498n,NaN
41164,магнитный_компас.n.1,компас.n.1,wn:03706229n,NaN
25825,gavidae.n.1,птица_семьи.n.1,wn:02048832n,NaN
627,туз_червей.n.2,туз.n.2,wn:02674273n,Червонный туз — это карта в колоде игральных к...
28835,фруктовый_бар.n.1,печенье.n.1,wn:07637737n,NaN


In [35]:
from ruwordnet import RuWordNet
wn = RuWordNet()

In [36]:
found = 0
valid_idx = []
for i, word in tqdm(enumerate(df1['Synset']), total=len(df1)):
    word = word.split('.')[0].replace('_', ' ')
    
    if len(wn.get_synsets(word)) > 0:
    # try:
    #     wn[word]
        
        valid_idx.append(i)
    # except KeyError:
    #     continue


  0%|          | 0/63346 [00:00<?, ?it/s]

100%|██████████| 63346/63346 [00:25<00:00, 2521.15it/s]


In [37]:
len(valid_idx)

15753

In [40]:
filtered_wn = df1.loc[valid_idx]

In [41]:
filtered_wn.sample(20)

,Synset,Hypernym,WordnetKey,Definition
25132,событие.n.1,физическое_явление.n.1,wn:11453860n,Собы́тие в теории относительности — моментальн...
21623,дисциплина.n.1,тренинг.n.1,wn:00893836n,Дисципли́на
36429,недопустимость.n.1,недопустимость.n.1,wn:04794408n,NaN
15524,восхождение.n.2,восходящий.n.2,wn:00325110n,NaN
29062,табун.n.1,агент_фосфорорганическими_нерва.n.1,wn:15066125n,Табу́н
43779,моцарелла.n.1,сыр.n.1,wn:07854184n,"Моцаре́лла, моццаре́лла"
19327,сокращение.n.4,сбережения.n.4,wn:00192910n,NaN
3665,антропология.n.1,общественные_науки.n.1,wn:06143546n,Антрополо́гия
20275,акт.n.1,юридический_документ.n.1,wn:06545137n,"Юридически оформленное дело, часто сделка"
13905,задание.n.1,задание.n.1,wn:00731222n,NaN


In [43]:
filtered_wn.index = list(range(len(filtered_wn)))

In [46]:
found = 0
valid_idx = []
for i, hyp in tqdm(enumerate(filtered_wn['Hypernym']), total=len(filtered_wn)):
    if pd.isna(hyp):
        continue
    for word in hyp.split('\t'):
        word = word.split('.')[0].replace('_', ' ')
        
        if len(wn.get_synsets(word)) > 0:
        # try:
        #     wn[word]
            
            valid_idx.append(i)
            found += 1
        # except KeyError:
        #     continue


  0%|          | 41/15753 [00:00<00:39, 400.76it/s]

100%|██████████| 15753/15753 [00:10<00:00, 1569.07it/s]


In [51]:
fil2 = filtered_wn.loc[valid_idx]
fil2.index = list(range(len(fil2)))

In [59]:
fil3 = fil2[(fil2['Synset'] != fil2['Hypernym'])]

In [61]:
len(fil3)

7961

In [60]:
out_path = 'test_ru_filter.edgelist'
G = nx.from_pandas_edgelist(fil3, 'Hypernym',
                                 'Synset', create_using=nx.DiGraph)

nodes_to_add = []
edges_to_add = []
nodes_to_delete = []

for node in G:
    if len(str(node).split('\t')) > 1:
        nodes_to_add.extend((str(node).split('\t')))
        edges_to_add.extend(create_all_edges(str(node).split('\t'), list(G[node])))
        nodes_to_delete.append(node)

G.add_nodes_from(nodes_to_add)
G.add_edges_from(edges_to_add)
G.remove_nodes_from(nodes_to_delete)

for bad_edge in tqdm(generator(G)):
    G.remove_edge(*bad_edge)

nx.write_edgelist(G, out_path, delimiter='\t')

35it [00:01, 21.04it/s]


In [87]:
a = a.replace('_', ' ')
lb = a.find('(')
rb = a.find(')')

print(a[:lb] + a[rb+1:])

пас 


In [82]:
wn.get_synsets('пас')

[Synset(id="127461-N", title="ПЕРЕДАЧА ПАСА")]

In [43]:
wn['apprisal'], wn['language'], wn['zombie']

([WNSense(key="apprisal%1:10:00::", name="apprisal")],
 [WNSense(key="language%1:10:03::", name="language"),
  WNSense(key="language%1:10:01::", name="language"),
  WNSense(key="language%1:10:00::", name="language"),
  WNSense(key="language%1:09:00::", name="language")],
 [WNSense(key="zombie%1:18:00::", name="zombie")])

In [15]:
df1[df1['Synset'].str.contains('азрег')]

,Synset,Hypernym,WordnetKey,Definition
5118,азрег.n.1,турецкие_денежной_единицы.n.1,wn:13687420n,NaN
10528,helix_азрегза.n.1,Helix_aspersa.n.1,wn:01945143n,NaN


In [16]:
df1[df1['Synset'].str.contains('зарегистрируйтесь')]

,Synset,Hypernym,WordnetKey,Definition
52119,зарегистрируйтесь_языке.n.1,тональный_язык.n.1,wn:06905243n,NaN


In [17]:
df1[df1['Synset'].str.contains('контур_языке')]

,Synset,Hypernym,WordnetKey,Definition
17452,контур_языке.n.1,тональный_язык.n.1,wn:06905141n,NaN


In [18]:
df1[df1['Synset'].str.contains('criollo')]

,Synset,Hypernym,WordnetKey,Definition
18773,criollo_людей.n.1,испаноамериканцы.n.1,wn:09978697n,NaN


In [19]:
df1[df1['Synset'].str.contains('dimensión')]

,Synset,Hypernym,WordnetKey,Definition
21344,dimensión.n.1,величина_(математика).n.1,wn:05093581n,NaN


In [52]:
df2 = df1[~df1['Definition'].isna()]

In [53]:
len(df2)

25434

In [21]:
df1[df1['Definition'].isna()].sample(10)

,Synset,Hypernym,WordnetKey,Definition
60526,племенное_общество.n.1,общество.n.1,wn:08288390n,NaN
20850,желание.n.3,NaN,wn:04945057n,NaN
4589,вооруженный_разбой.n.1,грабёж.n.1,wn:00781912n,NaN
54566,городского_управления.n.1,выборного_должностного_лица.n.1,wn:10576676n,NaN
54408,ожо́г_второ́й_сте́пени.n.3,ожог.n.3,wn:14291295n,NaN
51781,переоценка.n.1,NaN,wn:05747582n,NaN
24607,eragrostis.n.1,liliopsid_рода.n.1,wn:12119947n,NaN
23963,Emilia.n.1,asterid_двудольных_рода.n.1,wn:11963158n,NaN
16234,колледж_мальчик.n.1,студент.n.1,wn:09937056n,NaN
6233,вернуться_имеет_значения.n.6,NaN,wn:06390805n,NaN


In [26]:
df2

,Synset,Hypernym,WordnetKey,Definition
0,ноль.n.1,цифра.n.1,wn:13742358n,"Ноль — целое число, которое при сложении с люб..."
1,1_(число).n.1,цифра.n.1,wn:13742573n,"1 — наименьшее натуральное число, целое число ..."
2,додеканол.n.2,спирты.n.2,wn:14930670n,"Додеканол — органическое вещество, относится к..."
4,10_(число).n.1,большое_целое_число.n.1,wn:13746512n,"10 — натуральное число, расположенное между чи..."
5,100_(число).n.1,большое_целое_число.n.1,wn:13750415n,100 — натуральное и круглое число между 99 и 1...
...,...,...,...,...
63337,зуни_(народ).n.1,pueblo.n.1,wn:09673232n,"Зу́ни, зу́ньи"
63338,зурванизм.n.1,ересь.n.1\tбогословское_учение.n.1,wn:06251553n,Зурванизм — предположительно философское течен...
63339,зайдеко.n.1,кантри.n.1,wn:07062550n,За́йдеко (англ.
63344,зимаза.n.1,ферменты.n.1,wn:15109586n,Зимаза


In [23]:
G = nx.from_pandas_edgelist(df2, 'Hypernym', 'Synset', create_using=nx.DiGraph)

In [24]:
def create_all_edges(parents, children):
    res = []
    
    for parent in parents:
        for child in children:
            res.append((parent, child))

    return res

nodes_to_add = []
edges_to_add = []
nodes_to_delete = []

for node in G:
    if len(str(node).split('\t')) > 1:
        nodes_to_add.extend((str(node).split('\t')))
        edges_to_add.extend(create_all_edges(str(node).split('\t'), list(G[node])))
        nodes_to_delete.append(node)

In [25]:
G.add_nodes_from(nodes_to_add)
G.add_edges_from(edges_to_add)
G.remove_nodes_from(nodes_to_delete)

In [26]:
nx.find_cycle(G)

[('цифра.n.1', 'цифра.n.1')]

In [29]:
gen = generator()

In [31]:
next(gen)

('цифра.n.1', 'цифра.n.1')

In [32]:
for bad_edge in tqdm(generator()):
    G.remove_edge(*bad_edge)
    print(bad_edge)

0it [00:00, ?it/s]

2it [00:00, 13.51it/s]

('цифра.n.1', 'цифра.n.1')
('кран.n.1', 'кран.n.1')
('голубика.n.1', 'голубика.n.1')


6it [00:00, 12.73it/s]

('роза.n.1', 'роза.n.1')
('лещ.n.2', 'лещ.n.2')
('полынь_горькая.n.1', 'полынь_горькая.n.1')


...

('деформация.n.2', 'деформация.n.2')


98it [00:20,  3.60it/s]

('магнитофон.n.1', 'магнитофон.n.1')


99it [00:20,  4.82it/s]

('ксеродерма.n.1', 'ксеродерма.n.1')


In [92]:
create_all_edges(str(node).split('\t'), list(G[node]))

[('австрали́ец.n.1', 'Аборигены_Австралии.n.1'),
 ('этническая_группа.n.1', 'Аборигены_Австралии.n.1')]

In [48]:
df2['Hypernym'].apply(lambda x: len(str(x).split('\t'))).max()

5

In [93]:
G.remove_node(node)

In [115]:
nx.write_edgelist(G, 'test_graph.edgelist', delimiter='\t', data=True )

In [118]:
G['диаспора.n.2']

AtlasView({})

In [119]:
G = nx.read_edgelist("test_graph.edgelist", delimiter="\t", create_using=nx.DiGraph)

In [121]:
G.number_of_edges()

26087